In [ ]:
from selenium import webdriver as wb # 브라우저를 제어하기 위한 도구
from selenium.webdriver.common.keys import Keys # 키보드 제어
import time # 시간 딜레이를 위한 모듈
from bs4 import BeautifulSoup as bs
from datetime import timedelta
import datetime
from urllib.request import urlretrieve
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
import random
import json 
import pandas as pd

인스타그램 로그인 코드

In [ ]:
# 인스타그램 로그인 할 아이디와 비밀번호
login_id = "login_id"
login_pw = "login_pw"

In [ ]:
# 크롬드라이버 부르기
driver = wb.Chrome()

# 로그인 페이지로 이동
login_url = "https://www.instagram.com/accounts/login/"
driver.get(login_url)
time.sleep(2)
# 아이디 입력
id_form = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(1) > div > label > input')
time.sleep(2)
id_form.send_keys(login_id)
time.sleep(2)
# 비밀번호 입력
pw_form = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(2) > div > label > input')
pw_form.send_keys(login_pw)
time.sleep(2)
pw_form.send_keys(Keys.ENTER)
time.sleep(2)

해시태그 검색하기

In [ ]:
# 첫번째 사진 클릭
def first_img_click() :
    first_img_css="div.v1Nh3.kIKUG._bz0w" 
    driver.find_element_by_css_selector(first_img_css).click()

In [ ]:
next_arrow_btn_css1="._65Bje.coreSpriteRightPaginationArrow" 
next_arrow_btn_css2="._65Bje.coreSpriteRightPaginationArrow"
def next_img_click(error) :
    try :
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, next_arrow_btn_css1))) 
        driver.find_element_by_css_selector(next_arrow_btn_css2).click()  
        print("==== ",error,"로 인해 다음으로 넘어가기 ====")
        time.sleep(2)
        return(True)
    except :
        return(False)

In [ ]:
# 해시태그로 검색
keyword = "일상"
driver.get("https://www.instagram.com/explore/tags/"+keyword)
time.sleep(5)
first_img_click()

In [ ]:
upload_id_object_css="div.e1e1d > span.Jv7Aj.MqpiF > a.sqdOP.yWX7d._8A5w5.ZIAjV " 

In [ ]:
location_infos = [] 
location_hrefs = [] 
upload_ids = [] 
count_extract = 0 
# 몇개 수집할 것인지 정하기
wish_num = 100
check_arrow = True 

while True: 
    if count_extract > wish_num: 
        print("==== ",wish_num,"개 만큼 수집 완료 ====")
        break 
        time.sleep(5.0) 
        
    if check_arrow == False: 
        print("check_arrow == false")
        break 
        
        
    # 올린사람 ID  수집
    try: 
        upload_id_object = driver.find_element_by_css_selector(upload_id_object_css) 
        upload_id = upload_id_object.text 
        print(upload_id)
        upload_ids.append(upload_id) 
    except: 
        upload_id = None 
    
    
    # 다음으로 클릭
    try: 
        WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CSS_SELECTOR, next_arrow_btn_css1))) 
        driver.find_element_by_css_selector(next_arrow_btn_css2).click() 
        time.sleep(1)
                    
    except: 
        check_arrow = False 
                    
    count_extract += 1


In [ ]:
uk_upload_ids = []
for v in upload_ids:
    if v not in uk_upload_ids:
        uk_upload_ids.append(v)

수집하기

In [ ]:
today = datetime.datetime.now()
week_ago= today - timedelta(days = 7)
three_months_ago = today - timedelta(days = 30*3)

In [ ]:
profile_object_select = '.g47SY'
location_object_css="div.o-MQd.z8cbW > div.M30cS > div.JF9hh > a.O4GlU" 
date_object_css="div.k_Q0X.NnvRN > a.c-Yi7 > time._1o9PC.Nzb55" 
comment_click_css = ""
tag_css = '.XQXOT.pXf-y'
comment_more_btn='span.EizgU'
tag_css = '.XQXOT.pXf-y'
img_css = 'div > div > div.KL4Bh > img'
like_css = '.Nm9Fw span'
account_tag_css = 'div.xUdfV'
place_tag_css = 'a.O4GlU'

In [ ]:
check_arrow = True

In [ ]:
insta_data = []
cnt = 0
for user_id in uk_upload_ids :
    no_hashtag =0
    post_cnt = 0
    cnt += 1
    print("\n===",user_id," 크롤링 시작  (",cnt,"번째 ) ===")
    try:
        driver.get("https://www.instagram.com/"+user_id)
        profile_html = bs(driver.page_source,'html.parser')
        time.sleep(3)
        user_info = profile_html.select(profile_object_select)

        # 게시물 수, 팔로워수, 팔로우수
        user_total = int(user_info[0].text.replace(',',''))
        user_follower = int(user_info[1]['title'].replace(',',''))
        user_follow = int(user_info[2].text.replace(',',''))

        check_arrow = True
        count_extract = 1

        first_img_click()
        time.sleep(5)

        while True:
            
            count_extract += 1
            if user_follower>10000 :
                break

            # 다음 게시물이 없으면 멈추기
            if check_arrow == False: 
                print("==== ",user_id, "의 모든 게시물 크롤링 완료 ==== ")
                break 

            # 시간 수집
            try :
                date_object = driver.find_element_by_css_selector(date_object_css) 
                time.sleep(2)
                date_time = date_object.get_attribute("datetime")
                time_dt = datetime.datetime.strptime(date_time,'%Y-%m-%dT%H:%M:%S.%fZ') +  timedelta(hours = 9)  # 9시간 차이
                user_post_day = time_dt.strftime('%Y-%m-%d')
                user_post_time = time_dt.strftime('%H:%M:%S')
            except :
                    check_arrow = next_img_click("시간 수집") 
                    continue


            if(time_dt<three_months_ago) :
                print(user_id, " ==== 3개월보다 오래되어 break ====")
                break

            if not (time_dt<=week_ago and three_months_ago<=time_dt) : 
                check_arrow = next_img_click("시간 조건")
                continue       

            # 댓글 더보기 클릭
            try : 
                view = driver.find_elements_by_css_selector(comment_more_btn)
                for i in view :
                    i.click()        
                    time.sleep(0.5)
            except:
                check_arrow = next_img_click("댓글 더보기") 
                continue

            # 태그 수집
            try: 
                user_hash_tags = []
                time.sleep(3)
                data = driver.find_element_by_css_selector(tag_css) # C7I1f X7jCj 
                time.sleep(3)
                tag_raw = data.text 
                tag = re.findall('#[A-Za-z0-9가-힣]+', tag_raw) 
                if not tag :  # 해시태그가 하나도 없다면
                    check_arrow = next_img_click("해시태그 없음") 
                    no_hashtag+=1
                    if(no_hashtag >= 8) :
                        print("==== 해시태그 없는 게시물이 많아 break ====")
                        break # 해시태그 없는 게시물이 8번 정도 있으면 break
                    continue   # 크롤링을 멈춘다
                tag = ''.join(tag).replace("#"," ") # "#" 제거
                user_hash_tags = tag.split()
            except: 
                check_arrow = next_img_click("해시태그") 
                continue

            # 위치 수집
            try: 
                location_object = driver.find_element_by_css_selector(location_object_css) 
                user_place_tags = 1
                print(user_id, " == 위치 : " ,user_place_tags)
            except: 
                user_place_tags = 0 
                print(user_id, " === 위치 : " ,user_place_tags)


            # 계정  태그 수
            try:
                account_object = driver.find_elements_by_css_selector(account_tag_css)
                user_account_tags = len(account_object)
                print(user_id, " == 계정  태그 수 : " ,user_account_tags)
            except: 
                check_arrow = next_img_click("계정 태그")        
                continue



            # 좋아요 수집
            try:
                like_object = driver.find_element_by_css_selector(like_css)
                like = like_object.text
                user_like = int(like)
                print(user_id, " == 좋아요 수 : ", user_like)
            except:
                check_arrow = next_img_click("좋아요") 
                continue

            try:
                if(post_cnt > 10) :
                    print("==== 10개 수집 완료하여 break ====")
                    break
                img_object = driver.find_element_by_css_selector(img_css) 
                time.sleep(2)
                img_href = img_object.get_attribute("src") 
                img_name = '{}_{}.png'.format(user_id,count_extract)
                urlretrieve(img_href,'./insta_img/{}'.format(img_name))
                print(user_id, " ==== 사진 저장 성공 ====")
                post_cnt += 1
                
            except:  
                check_arrow = next_img_click("사진 수집") 
                continue

            user_data = [user_id, user_follower, user_follow, user_post_day, user_post_time, user_place_tags, user_account_tags, user_hash_tags,img_name,user_like]
            insta_data.append(user_data)

            print("\n===========결과===========")
            for i in user_data :
                print(i,end=" / ")
            print("\n=========================")
            # 다음으로 넘어가기
            check_arrow = next_img_click("수집 완료") 
    except:
            print("=== 비공개 계정으로 다음 계정으로 ===")
        
        


In [ ]:
insta_data_df = pd.DataFrame(insta_data, columns=['ID','팔로워','팔로우','업로드 날짜','업로드 시간','장소 태그 여부','계정 태그 여부','해시태그','사진 이름','좋아요 수'])
insta_data_df

In [ ]:
# csv로 저장하기
insta_data_df.to_csv("insta_train_{}.csv".format(keyword), index=False, encoding = 'euc-kr')